In [1]:
%matplotlib inline

import pyaudio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram

def play_sound(x, amp=0.5):

    p = pyaudio.PyAudio()
    # for paFloat32 sample values must be in range [-1.0, 1.0]
    stream = p.open(format=pyaudio.paFloat32,
                    channels=1,
                    rate=fs,
                    output=True)

    # play. May repeat with different volume values (if done interactively) 
    stream.write(amp*x)
    stream.stop_stream()
    stream.close()
    p.terminate()
    pass


def find_nearest(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx


def make_sound(f, fs=44100, duration=1.0):
    # generate samples, note conversion to float32 array
    x = (np.sin(2*np.pi*np.arange(fs*duration)*f/fs)).astype(np.float32)
    return x


def make_AM_sound(f, amp_modulator, fs=44100, duration=1.0):
    # generate samples, note conversion to float32 array
    x = make_sound(f, fs, duration)
    x = np.multiply(x, amp_modulator)
    return x


def make_FM_sound(f, freq_modulator, fs=44100, duration=1.0):
    # generate samples, note conversion to float32 array
    fs = float(fs)
    x = (np.sin(2*np.pi*np.arange(fs*duration)*np.multiply(f/fs, freq_modulator))).astype(np.float32)
    return x


def complex_tone(freqs, amps, duration):
   # make tone
    tones = list()
    for amp, f in zip(amps, freqs):
        tones.append(make_sound(f, fs=fs, duration=duration)*amp)
    tone = np.array(tones).sum(0)  # sum the frequencies together
    return tone


def power_to_db(power, conversion_val=10):
    '''
    This is the conversion from power -> dB, as taken from a stack exchange forum.
    '''
    return conversion_val*np.log10(power)


def db_to_power(db, conversion_val=10):
    '''
    This is the conversion from dB -> power, as also taken from a stack exchange forum.
    '''
    return 10 ** (db/conversion_val)


def get_fft_half(tone, fs):
    power = np.abs(np.fft.fft(tone))
    n = len(power)
    freq = np.fft.fftfreq(n, 1/float(fs))

    # just get first half of spectrum
    half_n = int(np.ceil(n/2.0))
#     power = (2.0 / n) * power[:half_n]
    power = power[:half_n]
    freq_half = freq[:half_n]
    return power, freq_half


def get_db(tone, fs):

    N = 8192  # not sure where this comes from ... :/
    win = np.hamming(N)                                                       
    x = tone[0:N] * win                             # Take a slice and multiply by a window

    sp = np.fft.rfft(x)                               # Calculate real FFT (this just means the first half)

    mag = np.abs(sp) 
    ref = np.sum(win) / 2                             # Reference : window sum and factor of 2
                                                      # because we are using half of FFT spectrum

    s_dbfs = 20 * np.log10(mag / ref)                 # Convert to dBFS
    
    freq = np.fft.fftfreq(N, 1/float(fs))
    half_n = int(np.ceil(N/2.0))  # not sure about N
    freq_half = freq[:half_n+1]
    
    return s_dbfs, freq_half


def plot_wave_and_fft(tone, fs, t_max, freq_max, power_max):
    # plot waveform
    f, axs = plt.subplots(1, 2, figsize=(15, 5))
    axs[0].plot(tone)
    axs[0].set_xlim([0, t_max])
    axs[0].set_xlabel('Time (ms)')
    axs[0].set_ylabel('Amplitude')

    # get fft
    power, freq_half = get_fft_half(tone, fs)

    # plot fft
    axs[1].plot(freq_half, power, lw=4, c='k')
    axs[1].set_xlim([0, freq_max])
    axs[1].set_ylim([0, power_max])
    axs[1].set_xlabel('Frequency (Hz)')
    axs[1].set_ylabel('Power')
    return f


def plot_wave_and_db(tone, fs, t_max, freq_max):
    # plot waveform
    f, axs = plt.subplots(1, 2, figsize=(15, 5))
    axs[0].plot(tone)
    axs[0].set_xlim([0, t_max])
    axs[0].set_xlabel('Time (ms)')
    axs[0].set_ylabel('Amplitude')

    # get fft
    db, freq_half = get_db(tone, fs)

    # plot fft
    axs[1].plot(freq_half, power_to_db(power), lw=4, c='k')
    axs[1].set_xlim([0, freq_max])
    axs[1].set_xlabel('Frequency (Hz)')
    axs[1].set_ylabel('dB')
    return f



In [48]:
import time

from scipy.io.wavfile import write
base_dir = '/Users/ellieabrams/Desktop/Projects/Shepard/exp/tones_final'
base_dir = '/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask'

# params
fs=44100  # sample freq
duration=0.8
tone_length = 0.3  # desired length in seconds

# test this freq range
min_freq = 200
max_freq = 600
n_freqs = ((max_freq - min_freq)*2)+1

# test this amp range
min_amp = 0.0000
max_amp = 0.05
n_amps = 50

# make tones, looping through each paramtere
for freq in np.linspace(min_freq, max_freq, n_freqs):
    for amps in np.linspace(min_amp, max_amp, n_amps):

        # make tone
        tone = complex_tone([freq], [amps], duration)
#         play_sound(tone)

        fname = '%s/%s-%s.wav' % (base_dir, freq, np.round(amps, 3))
        write(fname, fs, tone[0:int(fs*tone_length)])
        print(fname)

/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/200.0-0.0.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/200.0-0.001.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/200.0-0.002.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/200.0-0.003.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/200.0-0.004.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/200.0-0.005.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/200.0-0.006.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/200.0-0.007.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/200.0-0.008.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/200.0-0.009.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/200.0-

/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/200.5-0.046.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/200.5-0.047.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/200.5-0.048.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/200.5-0.049.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/200.5-0.05.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/201.0-0.0.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/201.0-0.001.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/201.0-0.002.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/201.0-0.003.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/201.0-0.004.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/201.0-0

/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/201.5-0.045.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/201.5-0.046.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/201.5-0.047.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/201.5-0.048.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/201.5-0.049.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/201.5-0.05.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/202.0-0.0.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/202.0-0.001.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/202.0-0.002.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/202.0-0.003.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/202.0-0

/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/203.5-0.014.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/203.5-0.015.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/203.5-0.016.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/203.5-0.017.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/203.5-0.018.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/203.5-0.019.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/203.5-0.02.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/203.5-0.021.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/203.5-0.022.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/203.5-0.023.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/203.5

/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/205.0-0.038.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/205.0-0.039.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/205.0-0.04.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/205.0-0.041.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/205.0-0.042.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/205.0-0.043.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/205.0-0.044.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/205.0-0.045.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/205.0-0.046.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/205.0-0.047.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/205.0

...

/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/595.5-0.031.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/595.5-0.032.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/595.5-0.033.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/595.5-0.034.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/595.5-0.035.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/595.5-0.036.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/595.5-0.037.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/595.5-0.038.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/595.5-0.039.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/595.5-0.04.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/595.5

/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/596.5-0.04.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/596.5-0.041.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/596.5-0.042.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/596.5-0.043.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/596.5-0.044.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/596.5-0.045.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/596.5-0.046.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/596.5-0.047.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/596.5-0.048.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/596.5-0.049.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/596.5

/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/598.0-0.008.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/598.0-0.009.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/598.0-0.01.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/598.0-0.011.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/598.0-0.012.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/598.0-0.013.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/598.0-0.014.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/598.0-0.015.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/598.0-0.016.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/598.0-0.017.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/598.0

/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/599.0-0.028.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/599.0-0.029.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/599.0-0.03.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/599.0-0.031.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/599.0-0.032.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/599.0-0.033.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/599.0-0.034.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/599.0-0.035.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/599.0-0.036.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/599.0-0.037.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/599.0

/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/600.0-0.027.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/600.0-0.028.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/600.0-0.029.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/600.0-0.03.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/600.0-0.031.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/600.0-0.032.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/600.0-0.033.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/600.0-0.034.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/600.0-0.035.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/600.0-0.036.wav
/Users/lauragwilliams/Documents/experiments/shepard/shepard/tones_pitchtask/600.0